In [ ]:
# default_exp readproc

# module name here

> API details.

In [3]:
#hide
from nbdev.showdoc import *



In [13]:
#export
import re
from pathlib import Path


def trim_cterm(inputFile, outputFile, motifToTrim):
    with open(inputFile, 'r') as inputFH:
        with open(outputFile,'w') as outputFH:
            i = 0
            for line in inputFH:
                i += 1
                if i%2 ==1: outputFH.write(line)
                else:
                    line = re.sub(motifToTrim, '', line)
                    #line += '\n'
                    outputFH.write(line)

Testing

In [16]:
test_input_path = Path.cwd()/'tests/test.fasta'
test_trimmed_output_path = Path.cwd()/'tests/test-out.fasta'
test_cterm_pattern_to_trim = 'LNEAIP.*$'

trim_cterm(test_input_path, test_trimmed_output_path, test_cterm_pattern_to_trim)

with open(test_trimmed_output_path, 'r') as test_FH:
    for i,line in enumerate(test_FH):
        if i == 0:
            assert line.rstrip() == '>123'
        if i == 1:
            assert line.rstrip() == 'CCC'
        if i == 2:
            assert line.rstrip() == '>128378'
        if i == 3:
            assert line.rstrip() == 'CCAC'
    
    assert i == 3